In [ ]:
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
from glob import glob
from PIL import Image
import time

# ================================
# 1. Data Loading & Preprocessing
# ================================

# Define the image size and channels
IMG_HEIGHT = 128
IMG_WIDTH = 128
IMG_CHANNELS = 3

def load_and_preprocess_image(image_path):
    """
    Load an image from path, resize it to (IMG_HEIGHT, IMG_WIDTH), 
    and normalize pixel values to [-1,1].
    """
    # Open the image using PIL.
    img = Image.open(image_path)
    
    # Convert to RGB
    img = img.convert('RGB')
    
    # Resize the image.
    img = img.resize((IMG_WIDTH, IMG_HEIGHT))
    
    # Convert image to numpy array.
    img_array = np.array(img).astype(np.float32)
    
    # Normalize to [-1,1]
    img_array = (img_array - 127.5) / 127.5
    
    return img_array

# List directories containing PNG files
data_dirs = [
    '/kaggle/input/cubicasa5k/cubicasa5k/cubicasa5k/colorful',
    # Add more directories as needed
]

# Gather file paths, excluding .svg files.
all_image_paths = []
for d in data_dirs:
    for root, dirs, files in os.walk(d):
        for filename in files:
            if filename.lower().endswith('.png'):
                full_path = os.path.join(root, filename)
                if os.path.exists(full_path):
                    all_image_paths.append(full_path)

print("Total valid images found:", len(all_image_paths))

# Create a tf.data.Dataset from the file paths.
def process_path(file_path):
    img = tf.numpy_function(func=load_and_preprocess_image, inp=[file_path], Tout=tf.float32)
    # Set the shape so TF knows the dimensions.
    img.set_shape([IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS])
    return img

dataset = tf.data.Dataset.from_tensor_slices(all_image_paths)
dataset = dataset.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
BATCH_SIZE = 64
dataset = dataset.shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# ================================
# 2. Define the DCGAN Architecture (Based on Reference)
# ================================

# Using TensorFlow's RandomNormal initializer for weights initialization
# This matches the initialization strategy in the reference code
initializer = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.02)

# Discriminator model
def make_discriminator_model():
    model = tf.keras.Sequential(name="discriminator")
    
    # First convolutional layer
    model.add(tf.keras.layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same', 
                                     input_shape=[IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS],
                                     kernel_initializer=initializer))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    
    # Second convolutional layer with batch normalization
    model.add(tf.keras.layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same',
                                     kernel_initializer=initializer))
    model.add(tf.keras.layers.BatchNormalization())
    
    # Third convolutional layer with batch normalization
    model.add(tf.keras.layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same',
                                     kernel_initializer=initializer))
    model.add(tf.keras.layers.BatchNormalization())
    
    # Fourth convolutional layer with batch normalization
    model.add(tf.keras.layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same',
                                     kernel_initializer=initializer))
    model.add(tf.keras.layers.BatchNormalization())
    
    # Fifth convolutional layer with batch normalization
    model.add(tf.keras.layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same',
                                     kernel_initializer=initializer))
    model.add(tf.keras.layers.BatchNormalization())
    
    # Flatten and output layer
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    
    return model

# Generator model
def make_generator_model():
    model = tf.keras.Sequential(name="generator")
    
    # Foundation for 4x4 image
    n_nodes = 256 * 4 * 4
    model.add(tf.keras.layers.Dense(n_nodes, input_dim=100, kernel_initializer=initializer))
    model.add(tf.keras.layers.Reshape((4, 4, 256)))
    
    # Upsample to 8x8
    model.add(tf.keras.layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', 
                                             activation='relu', kernel_initializer=initializer))
    
    # Upsample to 16x16
    model.add(tf.keras.layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', 
                                             activation='relu', kernel_initializer=initializer))
    
    # Upsample to 32x32
    model.add(tf.keras.layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', 
                                             activation='relu', kernel_initializer=initializer))
    
    # Upsample to 64x64
    model.add(tf.keras.layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', 
                                             activation='relu', kernel_initializer=initializer))
    
    # Upsample to 128x128
    model.add(tf.keras.layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', 
                                             activation='relu', kernel_initializer=initializer))
    
    # Output layer
    model.add(tf.keras.layers.Conv2D(3, (3, 3), activation='tanh', padding='same'))
    
    return model

# Create the models
generator = make_generator_model()
discriminator = make_discriminator_model()

# ================================
# 3. Define Losses and Optimizers
# ================================

# Using Adam optimizer with parameters from reference code
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)

# Loss functions
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=False)  # Changed to match sigmoid output

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

# ================================
# 4. Helper Functions
# ================================

# Generate latent points for generator input
def generate_latent_points(latent_dim, n_samples):
    return tf.random.normal([n_samples, latent_dim])

# Generate and save images
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)
    
    # Scale images from [-1,1] to [0,1]
    predictions = (predictions + 1) / 2.0
    
    fig = plt.figure(figsize=(4, 4))
    
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i])
        plt.axis('off')
    
    plt.suptitle(f"Epoch {epoch}")
    
    # Save the image
    filename = f"generated_epoch_{epoch}.png"
    plt.savefig(filename)
    plt.close()

# ================================
# 5. Training Step
# ================================

@tf.function
def train_step(images):
    noise = generate_latent_points(100, BATCH_SIZE)
    
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)
        
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)
        
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)
    
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    
    return gen_loss, disc_loss

# ================================
# 6. Training Loop
# ================================

def train(dataset, epochs):
    # Seed for consistent image generation
    seed = generate_latent_points(100, 16)
    
    for epoch in range(epochs):
        start = time.time()
        print(f"Starting epoch {epoch+1}/{epochs}...")
        
        batch_count = 0
        for image_batch in dataset:
            g_loss, d_loss = train_step(image_batch)
            batch_count += 1
            
            # Print batch progress
            if batch_count % 10 == 0:
                print(f">Epoch {epoch+1}, Batch {batch_count}, D_loss={d_loss:.3f}, G_loss={g_loss:.3f}")
        
        # Generate and save images
        generate_and_save_images(generator, epoch+1, seed)
        
        # Save the model periodically
        if (epoch + 1) % 10 == 0:
            generator.save(f'generator_model_epoch_{epoch+1}.h5')
            discriminator.save(f'discriminator_model_epoch_{epoch+1}.h5')
        
        print(f"Time for epoch {epoch+1} is {time.time()-start:.2f} sec")

# ================================
# 7. Run Training
# ================================

EPOCHS = 100
print("Starting training...")
train(dataset, EPOCHS)

Total valid images found: 700


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Starting training...
Starting epoch 1/100...
>Epoch 1, Batch 10, D_loss=0.299, G_loss=2.042
Time for epoch 1 is 121.63 sec
Starting epoch 2/100...
>Epoch 2, Batch 10, D_loss=0.110, G_loss=6.859
Time for epoch 2 is 109.05 sec
Starting epoch 3/100...
>Epoch 3, Batch 10, D_loss=0.128, G_loss=7.050
Time for epoch 3 is 105.87 sec
Starting epoch 4/100...
>Epoch 4, Batch 10, D_loss=0.695, G_loss=1.448
Time for epoch 4 is 106.26 sec
Starting epoch 5/100...
>Epoch 5, Batch 10, D_loss=0.590, G_loss=1.066
Time for epoch 5 is 105.85 sec
Starting epoch 6/100...
>Epoch 6, Batch 10, D_loss=0.065, G_loss=3.573
Time for epoch 6 is 107.60 sec
Starting epoch 7/100...
>Epoch 7, Batch 10, D_loss=0.167, G_loss=2.310
Time for epoch 7 is 105.90 sec
Starting epoch 8/100...
>Epoch 8, Batch 10, D_loss=0.757, G_loss=0.848
Time for epoch 8 is 142.91 sec
Starting epoch 9/100...
>Epoch 9, Batch 10, D_loss=0.192, G_loss=2.500
Time for epoch 9 is 105.15 sec
Starting epoch 10/100...
>Epoch 10, Batch 10, D_loss=0.219, G

In [ ]:
def test_generator(generator_model, num_samples=16):
    """
    Generate sample images using the trained generator.
    
    Args:
        generator_model: The trained generator model
        num_samples: Number of images to generate
    """
    print("Generating test samples...")
    
    # Generate random noise vectors
    noise = tf.random.normal([num_samples, 100])
    
    # Generate images
    generated_images = generator_model(noise, training=False)
    
    # Rescale images from [-1,1] to [0,1]
    generated_images = (generated_images + 1) / 2.0
    
    # Plot the generated images
    fig = plt.figure(figsize=(4, 4))
    for i in range(num_samples):
        plt.subplot(4, 4, i+1)
        plt.imshow(generated_images[i])
        plt.axis('off')
    
    plt.tight_layout()
    plt.savefig("test_generated_samples.png")
    plt.show()
    
    print("Test samples generated and saved as 'test_generated_samples.png'")

# Example usage:
# After training or loading a saved model:
test_generator(generator)